## Fit Strong Lens model with Lenstronomy

- author : Sylvie Dagoret-Campagne
- creation date : 2025-06-16
- last update : 2025-06-17

https://stackoverflow.com/questions/59699193/how-to-obtain-2d-cutout-of-an-image-from-a-skycoord-position

In [ ]:
# For Angle conversion
import matplotlib.pyplot as plt
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.visualization import simple_norm, ZScaleInterval,PercentileInterval

In [ ]:
from IPython.display import Image, display

## Configuration

### Target in ECDFS

- path to article : https://arxiv.org/pdf/1104.0931
- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

In [ ]:
fig_table1 = "input_figs/table1_gemcandidates.png"
fig_table2 = "input_figs/table2_gemcandidates.png"

In [ ]:
Image(url= fig_table1,width=800)

In [ ]:
Image(url= fig_table2,width=500)

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile ==> FindTileForCutoutGEM
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
# candidates
key = "ECDFS_G15422"
#key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]
tile_num = the_target["tile"]
coord = SkyCoord(ra=target_ra, dec=target_dec, unit=(u.deg, u.deg))

target_title = (
    the_target["field_name"] + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)

In [ ]:
tile_num

### Access to remote files

- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

### Config

- http://archive.stsci.edu/pub/hlsp/gems/v_mk1/h_gems_v35_mk1.fits
- url = f"http://archive.stsci.edu/pub/hlsp/gems/z_mk1/h_gems_z{tile_num}_mk1.fits"

In [ ]:
# select your bands
SELECT_VBAND = "v"
SELECT_ZBAND = "z"
SELECT_BAND = SELECT_ZBAND

#### define the cutoutfilename path

In [ ]:
cutout_inputpath = f"cutout_gems_{key}_b{SELECT_BAND}.fits"
cutout_inputpath_wt = f"cutout_gems_{key}_b{SELECT_BAND}_wt.fits"

### Load image and weight


🌈 Liste des stretch disponibles dans simple_norm
Tu peux utiliser ces chaînes de caractères comme valeur de stretch :
Valeur stretch	Description
- "linear"	Échelle linéaire simple (valeurs brutes entre min_cut et max_cut)
- "sqrt"	Échelle racine carrée – met en valeur les faibles intensités
- "power"	Échelle puissance (x^power)
- "log"	Échelle logarithmique – bon pour étendre la dynamique
- "asinh"	Échelle arcsinus hyperbolique – lisse, bien adaptée aux images astronomiques
- "sinh"	Hyperbolique sinus – moins courant
- "histeq"	Égalisation d'histogramme – utile pour répartir les contrastes

In [ ]:
# Calcul des coupures automatiques avec ZScale
#interval = ZScaleInterval()
#interval = PercentileInterval(98) 
#vmin_v, vmax_v = interval.get_limits(data_v)

# Normalisation avec stretch asinh
#norm_v = simple_norm(data_v, stretch='asinh', min_cut=vmin_v, max_cut=vmax_v)
#norm_v = simple_norm(data_v, 'asinh')

In [ ]:
def load_fits_image_with_norm(path):
    with fits.open(path) as hdul:
        data = hdul[0].data
        wcs = WCS(hdul[0].header)
        header = hdul[0].header
    vmin, vmax = ZScaleInterval().get_limits(data)
    norm = simple_norm(data, stretch='asinh', vmin=vmin, vmax=vmax)
    return data, wcs, norm, header

In [ ]:
# Charger les données

data, wcs, norm,header = load_fits_image_with_norm(cutout_inputpath)
data_wt, wcs_wt, norm_wt,header_wt = load_fits_image_with_norm(cutout_inputpath_wt)


fig = plt.figure(figsize=(12, 6))


ax1 = fig.add_subplot(1, 2, 1, projection=wcs)
im1 = ax1.imshow(data, origin='lower', cmap='gray', norm=norm)
ax1.set_title(f'Flux {key} {SELECT_BAND} band')
ax1.set_xlabel('RA')
ax1.set_ylabel('Dec')

ax2 = fig.add_subplot(1, 2, 2, projection=wcs_wt)
im2 = ax2.imshow(data_wt, origin='lower', cmap='gray', norm=norm_wt)
ax2.set_title(f'Weight {key} {SELECT_BAND}  band')
ax2.set_xlabel('RA')
ax2.set_ylabel('')

# Mise en page
plt.tight_layout()
plt.show()


## Header

In [ ]:
header

In [ ]:
exp_time = header["EXPTIME"]

### Work on coordinates and WCS

- the goal is to find the coordinates info and pixel scale info required by lenstropy

In [ ]:
from astropy.wcs.utils import proj_plane_pixel_scales

In [ ]:
wcs

In [ ]:
wcs.pixel_scale_matrix

In [ ]:
type(wcs)

In [ ]:
# Donne les échelles angulaires en degrés/pixel pour chaque axe
pixscale_deg = proj_plane_pixel_scales(wcs)  # [deg/pixel] en X et Y
pixscale_arcsec = pixscale_deg * 3600        # [arcsec/pixel]
print(f"Pixel scale : {pixscale_arcsec[0]:.5f} x {pixscale_arcsec[1]:.5f} arcsec/pixel")

In [ ]:
# pixel scale in arcsec
pixel_scale = np.average(pixscale_arcsec)

🧠 Détails techniques :
La matrice CD (coordinate description) est une matrice 2×2 qui donne la transformation linéaire (rotation, échelle, cisaillement) entre les pixels et les coordonnées en degrés (RA/Dec, etc).

Si l’en-tête FITS contient seulement PCi_j et CDELTi, la matrice CD est implicitement obtenue par :

$ CD = CDELT \times  PC $

Astropy fournit la méthode .get_cd() pour obtenir la matrice CD dans tous les cas.



In [ ]:
cd = wcs.wcs.cd  # si présent
if cd is None:
    # Sinon, recompose à partir de PC et CDELT
    cd = wcs.wcs.get_cd()

In [ ]:
cd

In [ ]:
print("Matrice CD (de pixels vers angles en degrés):")
print(cd)

In [ ]:
transform_pix2angle = cd*3600.
print("Matrice CD (de pixels vers angles en arcsec):")
print(transform_pix2angle)

In [ ]:
# Convertit le pixel (0, 0) en coordonnées du ciel (RA, Dec)
ra_at_xy_0, dec_at_xy_0 = wcs.wcs_pix2world(0, 0, 0)  # le 3e argument = 0 → origine '0-based' comme Python

print(f"RA: {ra_at_xy_0} deg, Dec: {dec_at_xy_0} deg")

## Plot sigma

In [ ]:
X = data.flatten()
Y = (1./np.sqrt(data_wt)).flatten()
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10, 4))
ax1.scatter(X, Y,marker="o",color="b",alpha=0.1)
ax1.set_yscale("log")
ax1.set_xscale("log")
ax1.set_xlim(0.00001,1)
ax1.set_ylim(0.028,0.06)
title = f'{key} {SELECT_BAND} band $\sigma_F$ vs Flux'
ax1.set_title(title)
ax1.set_xlabel("Flux")
ax1.set_ylabel("$\sigma_{Flux}$")

ax2.hist(Y,bins=50,histtype="step",color="b",lw=3)
ax2.set_title("standard deviation")
ax2.set_xlabel("$\sigma_{Flux}$")
cut_sigma = 0.042
ax2.axvline(cut_sigma,color="k",ls=":")

### In high sigma tail¶

In [ ]:
pixel_select = Y > cut_sigma

In [ ]:
X1 = X[pixel_select]
Y1 = Y[pixel_select]
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10, 4))
ax1.scatter(X1, Y1,marker="o",color="b",alpha=0.1)
ax1.set_yscale("log")
ax1.set_xscale("log")
ax1.set_xlim(0.00001,1)
#ax1.set_ylim(cut_sigma,0.06)
title = f'{key} {SELECT_BAND} band $\sigma_F$ vs Flux'
ax1.set_title(title)
ax1.set_xlabel("Flux")
ax1.set_ylabel("$\sigma_{Flux}$")

ax2.hist(Y1,bins=50,histtype="step",color="b",lw=3)
ax2.set_title("standard deviation")
ax2.set_xlabel("$\sigma_{Flux}$")

In [ ]:
F = data
sig = (1./np.sqrt(data_wt))

In [ ]:
Fsel = np.where(sig>cut_sigma, F,0.0)

In [ ]:
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(1, 2, 1, projection=wcs)
im1 = ax1.imshow(data, origin='lower', cmap='gray', norm=norm)
ax1.set_title(f'Flux {key} {SELECT_BAND} band')
ax1.set_xlabel('RA')
ax1.set_ylabel('Dec')

ax2 = fig.add_subplot(1, 2, 2, projection=wcs_wt)
im2 = ax2.imshow(Fsel, origin='lower', cmap='gray', norm=norm)
ax2.set_title(f'Flux sel {key} {SELECT_BAND}  band')
ax2.set_xlabel('RA')
ax2.set_ylabel('')

# Mise en page
plt.tight_layout()
plt.show()

### Remove high sigma tail

In [ ]:
pixel_select = Y < cut_sigma

In [ ]:
X1 = X[pixel_select]
Y1 = Y[pixel_select]
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10, 4))
ax1.scatter(X1, Y1,marker="o",color="b",alpha=0.1)
ax1.set_yscale("log")
ax1.set_xscale("log")
ax1.set_xlim(0.00001,1)
#ax1.set_ylim(cut_sigma,0.06)
title = f'{key} {SELECT_BAND} band $\sigma_F$ vs Flux'
ax1.set_title(title)
ax1.set_xlabel("Flux")
ax1.set_ylabel("$\sigma_{Flux}$")

ax2.hist(Y1,bins=50,histtype="step",color="b",lw=3)
ax2.set_title("standard deviation")
ax2.set_xlabel("$\sigma_{Flux}$")

### Check removed high sigma tail

In [ ]:
F = data
sig = (1./np.sqrt(data_wt))

In [ ]:
Fsel = np.where(sig<cut_sigma, F,0.0)

In [ ]:
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(1, 2, 1, projection=wcs)
im1 = ax1.imshow(data, origin='lower', cmap='gray', norm=norm)
ax1.set_title(f'Flux {key} {SELECT_BAND} band')
ax1.set_xlabel('RA')
ax1.set_ylabel('Dec')

ax2 = fig.add_subplot(1, 2, 2, projection=wcs_wt)
im2 = ax2.imshow(Fsel, origin='lower', cmap='gray', norm=norm)
ax2.set_title(f'Flux sel {key} {SELECT_BAND}  band')
ax2.set_xlabel('RA')
ax2.set_ylabel('')

# Mise en page
plt.tight_layout()
plt.show()

### LensModel module

 $\texttt{LensModel}$ module
$\texttt{LensModel}$ and its sub-packages execute all the purely lensing related tasks of *lenstronomy*. This includes ray-shooting, solving the lens equation, arrival time computation and non-linear solvers to optimize lens models for specific image configurations. The module allows consistent integration with single and multi plane lensing and an arbitrary superpositions of lens models. There is a wide range of lens models available. For details we refer to to the online-documentation.

### Single plane lensing
As an example of initializing a single plane lens model as a super-position of an elliptical power-law potential and a singular isothermal sphere mass distribution and execute some basic routines:

In [ ]:
# import the LensModel class #
from lenstronomy.LensModel.lens_model import LensModel

# specify the choice of lens models #
lens_model_list = ['EPL', 'SHEAR']

# setup lens model class with the list of lens models #
lensModel = LensModel(lens_model_list=lens_model_list)

# define parameter values of lens models #
#kwargs_spep = {'theta_E': 1.1, 'e1': 0.1, 'e2': 0.1, 'gamma': 2., 'center_x': 0.0, 'center_y': 0}
kwargs_spep = {'theta_E': 1.1, 'e1': 0.1, 'e2': -0.1, 'gamma': 2., 'center_x': 0.0, 'center_y': 0}
kwargs_shear = {'gamma1': -0.01, 'gamma2': .03}
kwargs_lens = [kwargs_spep, kwargs_shear]


# image plane coordinate #
theta_ra, theta_dec = target_ra, target_dec

# source plane coordinate #
beta_ra, beta_dec = lensModel.ray_shooting(theta_ra, theta_dec, kwargs_lens)
# Fermat potential #
fermat_pot = lensModel.fermat_potential(x_image=theta_ra, y_image=theta_dec, x_source=beta_ra, y_source=beta_dec, kwargs_lens=kwargs_lens)

# Magnification #
mag = lensModel.magnification(theta_ra, theta_dec, kwargs_lens)

In [ ]:
from lenstronomy.Plots import lens_plot
f, axex = plt.subplots(1, 1, figsize=(5, 5), sharex=False, sharey=False)
lens_plot.lens_model_plot(axex, lensModel=lensModel, kwargs_lens=kwargs_lens, sourcePos_x=beta_ra, sourcePos_y=beta_dec, point_source=True, with_caustics=True, fast_caustic=True, coord_inverse=False)
#lens_plot.lens_model_plot(axex[1], lensModel=lensModel_mp, kwargs_lens=kwargs_lens_mp, sourcePos_x=beta_ra, sourcePos_y=beta_dec, point_source=True, with_caustics=True, fast_caustic=True, coord_inverse=False)
f.show()

#### $\texttt{LightModel}$ module

The $\texttt{LightModel}$ class provides the functionality to describe galaxy surface brightnesses. $\texttt{LightModel}$ supports various analytic profiles as well as representations in shapelet basis sets. Any superposition of different profiles is allowed.

The parameter levelig the amplitude of the surface brightness of a certain profile is named 'amp'. The units are not further specified and are effectively reflecting a surface brightness quantitiy integrated over the unit of angle square. In the $\texttt{SimulationAPI}$ module, the user can conveniently chose astronomical magnitudes as inputs to the profiles, provided the magnitude zero point is declared. Have a look at the specific notebook to see a demonstration.

In [ ]:
# import the LightModel class #
from lenstronomy.LightModel.light_model import LightModel
# set up the list of light models to be used #
source_light_model_list = ['SERSIC']
lightModel_source = LightModel(light_model_list=source_light_model_list)
lens_light_model_list = ['SERSIC_ELLIPSE']
lightModel_lens = LightModel(light_model_list=lens_light_model_list)
# define the parameters #
kwargs_light_source = [{'amp': 100, 'R_sersic': 0.1, 'n_sersic': 1.5, 'center_x': beta_ra, 'center_y': beta_dec}]
import lenstronomy.Util.param_util as param_util
e1, e2 = param_util.phi_q2_ellipticity(phi=0.5, q=0.7)
kwargs_light_lens = [{'amp': 1000, 'R_sersic': 0.1, 'n_sersic': 2.5, 'e1': e1, 'e2': e2, 'center_x': 0.1, 'center_y': 0}]

# evaluate surface brightness at a specific position #
flux = lightModel_lens.surface_brightness(x=1, y=1, kwargs_list=kwargs_light_lens)

### $\texttt{PointSource}$ module
To accurately predict and model the position and flux of point sources, one has to apply different numerical procedures than for extended surface brightness features. The $\texttt{PointSource}$ class manages the different options in describing point sources (e.g. in the image plane or source plane, with fixed magnification or allowed with individual variations thereof) and provides a homogeneous interface to access image positions and magnifications.

In [ ]:
# import the PointSource class #
from lenstronomy.PointSource.point_source import PointSource

# unlensed source position #
point_source_model_list = ['SOURCE_POSITION']
pointSource = PointSource(point_source_type_list=point_source_model_list, lens_model=lensModel, fixed_magnification_list=[True])
kwargs_ps = [{'ra_source': beta_ra, 'dec_source': beta_dec, 'source_amp': 100}]
# return image positions and amplitudes #
x_pos, y_pos = pointSource.image_position(kwargs_ps=kwargs_ps, kwargs_lens=kwargs_lens)
point_amp = pointSource.image_amplitude(kwargs_ps=kwargs_ps, kwargs_lens=kwargs_lens)

# lensed image positions (solution of the lens equation) #
point_source_model_list = ['LENSED_POSITION']
pointSource = PointSource(point_source_type_list=point_source_model_list, lens_model=lensModel, fixed_magnification_list=[False])
kwargs_ps = [{'ra_image': theta_ra, 'dec_image': theta_dec, 'point_amp': np.abs(mag)*30}]
# return image positions and amplitudes #
x_pos, y_pos = pointSource.image_position(kwargs_ps=kwargs_ps, kwargs_lens=kwargs_lens)
point_amp = pointSource.image_amplitude(kwargs_ps=kwargs_ps, kwargs_lens=kwargs_lens)

## To do the fit

In [ ]:
noise_map = 1./np.sqrt(data_wt)

#### Make the PSF

La caméra ACS/WFC (Wide Field Channel) du Hubble Space Telescope a une échelle angulaire (pixel scale) de :

- ACS/WFC : ~0.0495 arcsec/pixel
- ACS/HRC (High Resolution Channel, maintenant inactive) : ~0.028 arcsec/pixel

In [ ]:
from lenstronomy.Data.psf import PSF

#psf_class = PSF(psf_type='PIXEL', kernel_point_source=psf_kernel)
psf_class = PSF(psf_type='GAUSSIAN', fwhm=0.11, pixel_size=pixel_scale)

In [ ]:
# 2. Définir la PSF comme gaussienne approximative

#psf_class = PSF(psf_type='PIXEL', kernel_point_source=psf_kernel)
# FWHM typique en z-band (ACS) : 0.11 arcsec
fwhm = 0.11  # arcsec
psf_class = PSF(psf_type='GAUSSIAN', fwhm=fwhm, pixel_size=pixel_scale)

In [ ]:
# kwargs pour la PSF
kwargs_psf = {
    'psf_type': 'GAUSSIAN',
    'fwhm': 0.11,
    'pixel_size': pixel_scale
}

In [ ]:
from lenstronomy.Data.imaging_data import ImageData

In [ ]:
# --- Exemple ---
kwargs_data = {
    'image_data': data,
    'noise_map': noise_map,
    'exposure_time': None,
    'background_rms': None,
    'exposure_time': exp_time,
    'ra_at_xy_0': ra_at_xy_0,  
    'dec_at_xy_0': dec_at_xy_0,
    'transform_pix2angle': transform_pix2angle 
}

In [ ]:
# 3. Définir l'objet ImageData

# --- Créer l'objet ImageData ---
data_class = ImageData(**kwargs_data)

In [ ]:
from lenstronomy.ImSim.image_linear_solve import ImageLinearFit
image_linear_fit = ImageLinearFit(data_class=data_class, psf_class=psf_class, lens_model_class=lensModel,
                        source_model_class=lightModel_source,
                        lens_light_model_class=lightModel_lens,
                        point_source_class=pointSource, 
                        kwargs_numerics=kwargs_numerics)

# we do not require the knowledge of the linear parameters #
del kwargs_light_source[0]['amp']
# apply the linear inversion to fit for the noisy image #
image_reconstructed, _, _, _ = image_linear_fit.image_linear_solve(kwargs_lens=kwargs_lens, kwargs_source=kwargs_light_source, 
                              kwargs_lens_light=kwargs_light_lens, kwargs_ps=kwargs_ps)

### $\texttt{Sampling}$ Module
The $\texttt{Sampling}$ Module consists of three major classes: 

- The $\texttt{Likelihood}$ class manages the specific likelihood function, consisting of the imaging likelihood and potential other data and constraints.
- The $\texttt{Param}$ class handles all the model choices and the parameters going in it. These classes handle all the model choices of the user and mitigate them to the external modules and from the external modules back to *lenstronomy*.
- The $\texttt{Sampler}$ class gives specific examples how the $\texttt{Likelihood}$ and $\texttt{Param}$ classes can be used to execute specific samplers. This last class can also be replaced by other sampling methods by the user.


#### $\texttt{Param}$ class
The $\texttt{Param}$ class is the interface of the *lenstronomy* conventions of parameters (list of keyword arguments) and the fitting conventions of $\texttt{CosmoHammer}$ or $\texttt{emcee}$ (single array consisting all parameters). The $\texttt{Param}$ class enables the user to set further options:

- keep certain parameters fixed
- handling of the linear parameters
- provide additional constraints on the modelling (e.g. fix source profile to point source position etc)

With the $\texttt{Param}$ class, lenstronomy can be integrated robustly with other sampling methods.




<div class="alert alert-warning">

**ATTENTION:** lenstronomy, by default, solves for the amplitude values of surface brighthness functions (in the lens and source) specified in the model by 'amp' through a linear minimization at each likelihood evaluation step. Solving a linear equation is more efficient than sampling the parameters in the non-linear exploration. Hence, the output of a sampling chain (be it PSO, MCMC or any other sampler used) do not contain the linear amplitudes and are set as =0.
To retriev the linear amplitude parameters, you have to re-do the linear inversion in post processing. This can be done with the $\texttt{LensModelPlot}$ class (see block further down) or with the $\texttt{ImageReconstruction}$ class in the $\texttt{Analysis}$ sub-package.

</div>

### The Models

#### $\texttt{LightModel}$ module

The $\texttt{LightModel}$ class provides the functionality to describe galaxy surface brightnesses. $\texttt{LightModel}$ supports various analytic profiles as well as representations in shapelet basis sets. Any superposition of different profiles is allowed.

The parameter levelig the amplitude of the surface brightness of a certain profile is named 'amp'. The units are not further specified and are effectively reflecting a surface brightness quantitiy integrated over the unit of angle square. In the $\texttt{SimulationAPI}$ module, the user can conveniently chose astronomical magnitudes as inputs to the profiles, provided the magnitude zero point is declared. Have a look at the specific notebook to see a demonstration.

In [ ]:
# import the LightModel class #
from lenstronomy.LightModel.light_model import LightModel
# set up the list of light models to be used #
source_light_model_list = ['SERSIC']
lightModel_source = LightModel(light_model_list=source_light_model_list)
lens_light_model_list = ['SERSIC_ELLIPSE']
lightModel_lens = LightModel(light_model_list=lens_light_model_list)
# define the parameters #
kwargs_light_source = [{'amp': 100, 'R_sersic': 0.1, 'n_sersic': 1.5, 'center_x': beta_ra, 'center_y': beta_dec}]
import lenstronomy.Util.param_util as param_util
e1, e2 = param_util.phi_q2_ellipticity(phi=0.5, q=0.7)
kwargs_light_lens = [{'amp': 1000, 'R_sersic': 0.1, 'n_sersic': 2.5, 'e1': e1, 'e2': e2, 'center_x': 0.1, 'center_y': 0}]

# evaluate surface brightness at a specific position #
flux = lightModel_lens.surface_brightness(x=1, y=1, kwargs_list=kwargs_light_lens)

In [ ]:
# import the parameter handling class #
from lenstronomy.Sampling.parameters import Param
# set options for constraint parameters #
# Here we join the position of the lens light and mass center. 
# There are more options available in the Param() class.
kwargs_constraints = {'joint_lens_with_light': [[0, 0, ['center_x', 'center_y']]]}

# list fixed parameters #
kwargs_fixed_lens = [{'gamma': 2.}, {'ra_0': 0, 'dec_0': 0}, {}]
kwargs_fixed_source = [{}]
kwargs_fixed_lens_light = [{}]

# optional: list lower and upper bounds in parameter space, otherwise it will be using lenstronomy default values! #
# list lower bounds in parameter space #
kwargs_lower_lens = [{'theta_E': 0, 'e1': -0.5, 'e2': -0.5, 'gamma': 1.5, 'center_x': -10., 'center_y': -10},
                     {'gamma1': -0.5, 'gamma2': -0.5}]
kwargs_lower_source = [{'R_sersic': 0.001, 'n_sersic': .5, 'center_x': -10, 'center_y': -10}]
kwargs_lower_lens_light = [{'R_sersic': 0.001, 'n_sersic': 0.5, 'e1': -0.5, 'e2': -0.5, 'center_x': -10, 'center_y': -10}]
kwargs_lower_ps = None

# list upper bounds in parameter space #
kwargs_upper_lens = [{'theta_E': 10, 'e1': 0.5, 'e2': 0.5, 'gamma': 2.5, 'center_x': 10., 'center_y': 10},
                     {'gamma1': 0.5, 'gamma2': 0.5}]
kwargs_upper_source = [{'R_sersic': 10, 'n_sersic': 5., 'center_x': 10, 'center_y': 10}]
kwargs_upper_lens_light = [{'R_sersic': 10, 'n_sersic': 5., 'e1': 0.5, 'e2': 0.5, 'center_x': 10, 'center_y': 10}]
kwargs_upper_ps = None

# make instance of parameter class with given model options, constraints and fixed parameters #
param = Param(kwargs_model, kwargs_fixed_lens, kwargs_fixed_source,
              kwargs_fixed_lens_light, kwargs_fixed_ps=None, 
              kwargs_lower_lens=kwargs_lower_lens, kwargs_lower_source=kwargs_lower_source,
              kwargs_lower_lens_light=kwargs_lower_lens_light, kwargs_lower_ps=kwargs_lower_ps,
              kwargs_upper_lens=kwargs_upper_lens, kwargs_upper_source=kwargs_upper_source,
              kwargs_upper_lens_light=kwargs_upper_lens_light, kwargs_upper_ps=kwargs_upper_ps,
              kwargs_lens_init=kwargs_lens, **kwargs_constraints)

# the number of non-linear parameters and their names #
num_param, param_list = param.num_param()

# parameter array for fitting routine #
param_array = param.kwargs2args(kwargs_lens, kwargs_light_source, kwargs_light_lens, kwargs_ps)

# recover keyword arguments list for lenstronomy from a parameter array of the fitting routine #
kwargs_out = param.args2kwargs(param_array)

# print settings #
param.print_setting()


In [ ]:
# Définition des listes de modèles (juste des listes de chaînes)
lens_model_list = ['SIE']
source_model_list = ['SERSIC_ELLIPSE']
lens_light_model_list = ['SERSIC_ELLIPSE']

# Tu prépares kwargs_model
kwargs_model = {
    'lens_model_list': lens_model_list,
    'source_light_model_list': source_model_list,
    'lens_light_model_list': lens_light_model_list
}

# Paramètres initiaux (listes de dict)
kwargs_lens = [{'theta_E': 1.0, 'e1': 0.1, 'e2': 0.05, 'center_x': 0.0, 'center_y': 0.0}]
kwargs_source = [{'amp': 1.0, 'R_sersic': 0.5, 'n_sersic': 1.5, 'e1': 0.1, 'e2': 0.05,
                  'center_x': 0.1, 'center_y': 0.1}]
kwargs_lens_light = [{'amp': 1.0, 'R_sersic': 0.6, 'n_sersic': 4.0, 'e1': 0.0, 'e2': 0.0,
                      'center_x': 0.0, 'center_y': 0.0}]

kwargs_params = {
    'lens_model': kwargs_lens,
    'source_model': kwargs_source,
    'lens_light_model': kwargs_lens_light
}



In [ ]:
#from lenstronomy.Util import param_util
#from lenstronomy.Workflow.fitting_sequence import FittingSequence
#from lenstronomy.Plots.model_plot import ModelPlot

In [ ]:
# Il faut un dictionnaire kwargs_likelihood, exemple simple :
kwargs_likelihood = {
    'check_bounds': True,       # Vérifie que les paramètres restent dans les bornes
    'image_likelihood_mask_list': None  # Si tu veux un masque sur l’image, sinon None
}

In [ ]:
# Puis tu peux initialiser ta FittingSequence
from lenstronomy.Workflow.fitting_sequence import FittingSequence

fitting_seq = FittingSequence(
    kwargs_data,
    kwargs_psf,
    kwargs_model,
    kwargs_likelihood,
    kwargs_params
)


In [ ]:
fitting_list = [
    ['PSO', {'n_particles': 100, 'n_iterations': 200}]
]

result = fitting_seq.fit_sequence(fitting_list)